## 학습 마친 모델 실전 투입

- 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 인퍼런스 설정

- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름
  * 단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 함
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치
- max_seq_length : 토큰 기준 입력 문장 최대 길이, 아무 것도 입력하지 않으면 64

In [6]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-paircls1",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-paircls1/epoch=0-val_loss=0.86.ckpt


### 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어들이기

- 체크포인트 로드

In [7]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

- BERT 설정 로드
  * 파인튜닝한 모델이 사용한 프리트레인 마친 언어 모델의 설정 값들을 읽어 들일 수 있음

In [8]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(), 
)

- BERT 모델 초기화
  * 초기화한 BERT 모델에 체크포인트(fine_tuned_model_ckpt)를 읽어들임

In [9]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)  

- 체크포인트 주입

In [10]:
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

- 평가 모드로 전환
  * 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할

In [11]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

- 전체 코드

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),  #텐서 크기 계산
)

model = BertForSequenceClassification(pretrained_model_config) #bert 모델 초기화
model.load_state_dict({k.replace("model.",""): v fork k, v in fine_tuned_model_ckpt['state_dict'].items()}) #체크포인트 주입
model.eval()

- 토크나이저 로드
  * 토크나이저 초기화

In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### 인퍼런스 함수 선언


- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 생성
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력
- 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태
  * 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꿈
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만듦


In [29]:
def inference_fn(premise, hypothesis):
  inputs = tokenizer(
      [(premise, hypothesis)],
      max_length=args.max_seq_length,
      padding="max_length",
      truncation=True,     #전제와 가설을 모델 입력값으로 만들기
  )
  with torch.no_grad():
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) #inputs를 파이토치 텐서로 바꾼 뒤 모델 계산하기
    prob = outputs.logits.softmax(dim=1)  #로짓에 소프트맥스 취하기
    entailment_prob = round(prob[0][0].item(), 2) 
    contradiction_prob = round(prob[0][1].item(), 2) 
    neutral_prob = round(prob[0][2].item(), 2)    #확률을 소수점 2자리로 반올림
    if torch.argmax(prob) == 0:
      pred = "참 (entailment)"
    elif torch.argmax(prob) == 1:
      pred = "거짓 (contradiction)"
    else:
      pred = "중립 (neutral)"    #예측 확률의 최댓값 위치에 따라 pred 만들기

  return {
      'premise': premise,
      'hypothesis': hypothesis,
      'prediction': pred,
      'entailment_data': f"참 {entailment_prob}",
      'contradiction_data': f"거짓 {contradiction_prob}",      
      'neutral_data': f"중립 {neutral_prob}",       
      'entailment_width': f"참 {entailment_prob*100}%",
      'contradiction_width': f"거짓 {contradiction_prob*100}%",      
      'neutral_width': f"중립 {neutral_prob*100}%", 
  }

In [30]:
premise = '오늘은 금요일이다.'
hypothesis = '내일은 월요일이다.'
inference_fn(premise, hypothesis)

{'premise': '오늘은 금요일이다.',
 'hypothesis': '내일은 월요일이다.',
 'prediction': '거짓 (contradiction)',
 'entailment_data': '참 0.13',
 'contradiction_data': '거짓 0.72',
 'neutral_data': '중립 0.16',
 'entailment_width': '참 13.0%',
 'contradiction_width': '거짓 72.0%',
 'neutral_width': '중립 16.0%'}

In [31]:
premise = '나는 오늘 공원에서 친구를 만났다.'
hypothesis = '나는 오늘 공원에 갔다.'
inference_fn(premise, hypothesis)

{'premise': '나는 오늘 공원에서 친구를 만났다.',
 'hypothesis': '나는 오늘 공원에 갔다.',
 'prediction': '참 (entailment)',
 'entailment_data': '참 0.79',
 'contradiction_data': '거짓 0.07',
 'neutral_data': '중립 0.14',
 'entailment_width': '참 79.0%',
 'contradiction_width': '거짓 7.000000000000001%',
 'neutral_width': '중립 14.000000000000002%'}

### 웹 서비스 

#### 만들기 준비



- `ngrok`: 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구
  * `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 
  * 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행
```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```
  * 인증 토큰: 2K7HpOJzbZrOEtWlKir6NjCmzOj_7oQuKdBKnQ1fkwDuXUW8o


In [32]:
!rm -rf /root/.ngrok2

In [33]:
!mkdir /root/.ngrok2 && echo "authtoken: 2K7HpOJzbZrOEtWlKir6NjCmzOj_7oQuKdBKnQ1fkwDuXUW8o" > /root/.ngrok2/ngrok.yml

In [34]:
!ls /root -al

total 64
drwx------ 1 root root 4096 Jan 11 01:54 .
drwxr-xr-x 1 root root 4096 Jan 11 01:33 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jan 11 01:19 .cache
drwx------ 1 root root 4096 Jan 11 01:18 .config
drwxr-xr-x 5 root root 4096 Jan  9 14:51 .ipython
drwx------ 2 root root 4096 Jan  9 14:51 .jupyter
drwxr-xr-x 2 root root 4096 Jan  9 14:48 .keras
drwxr-xr-x 1 root root 4096 Jan  9 14:51 .local
drwxr-xr-x 2 root root 4096 Jan 11 01:54 .ngrok2
drwxr-xr-x 4 root root 4096 Jan  9 14:51 .npm
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jan  9 14:51 .wget-hsts


#### 개시
- 아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듦
- 플라스크(flask)라는 파이썬 라이브러리의 도움을 받아 웹 서비스를 띄움

In [35]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2f54-35-236-168-79.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 01:54:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 01:54:52] "GET /favicon.ico HTTP/1.1" 404 -
